## Cálculo da distância percorrida

Nesse notebook é calculada a distância percorrida entre os pontos e a emissão de poluentes

In [1]:
import pandas as pd
import random
from utils.save_csv import save_csv
from utils.load_csv import load_csv

In [2]:
consumo_diesel = load_csv('consumo_diesel.csv')

fatores_emissao = load_csv('fatores_emissao.csv')

veiculos = load_csv('veiculos.csv')

posicoes_df = load_csv('posicoes_onibus.csv')

In [3]:
consumo_diesel.head()

,tecnologia,sem_ar_l_km,sem_ar_kg_km,com_ar_l_km,com_ar_kg_km
0,Miniônibus,0.30,0.252,0.35,0.294
1,Midiônibus,0.40,0.336,0.47,0.395
2,Básico,0.46,0.386,0.53,0.445
3,Padron,0.55,0.462,0.63,0.529
4,Articulado (18m),0.71,0.596,0.80,0.672


In [4]:
len(consumo_diesel)

7

In [5]:
consumo_diesel['emissa_no2_l_diesel'] = [random.uniform(0.02, 0.05) for _ in range(len(consumo_diesel))]
consumo_diesel['emissa_co2_l_diesel'] = [random.uniform(0.05, 0.1) for _ in range(len(consumo_diesel))]

In [6]:
consumo_diesel.head()

,tecnologia,sem_ar_l_km,sem_ar_kg_km,com_ar_l_km,com_ar_kg_km,emissa_no2_l_diesel,emissa_co2_l_diesel
0,Miniônibus,0.30,0.252,0.35,0.294,0.038557,0.086991
1,Midiônibus,0.40,0.336,0.47,0.395,0.021333,0.089260
2,Básico,0.46,0.386,0.53,0.445,0.021932,0.085327
3,Padron,0.55,0.462,0.63,0.529,0.034012,0.094641
4,Articulado (18m),0.71,0.596,0.80,0.672,0.033366,0.067354


In [7]:
del fatores_emissao

In [8]:
veiculos.head()

,id_veiculo,tipo_veiculo,modelo,fase_emissao,eletrico
0,1001,diesel,Biarticulado,CONAMA P8 (EURO VI),False
1,1002,diesel,Midiônibus,CONAMA P7 (EURO V),False
2,1003,eletrico,Básico,CONAMA P5 (EURO III),True
3,1004,eletrico,Articulado (23m),CONAMA P8 (EURO VI),True
4,1005,eletrico,Articulado (23m),CONAMA P8 (EURO VI),True


In [9]:
posicoes_df.head()

,id_onibus,x,y,t
0,1001,-46.591957,-23.540093,2025-08-12T12:22:04.509539
1,1001,-46.584857,-23.560791,2025-08-12T11:55:04.509539
2,1001,-46.613103,-23.502845,2025-08-12T11:57:04.509539
3,1001,-46.555077,-23.584262,2025-08-12T12:31:04.509539
4,1001,-46.622384,-23.545054,2025-08-12T11:37:04.509539


In [10]:
def get_tipo_onibus(id_onibus):

    onibus = veiculos[veiculos['id_veiculo']==id_onibus]

    return onibus['modelo'].unique()[0]

In [11]:
def get_fator_consumo(tipo_veiculo):

    cosumo_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]

    return cosumo_modelo['com_ar_l_km'].unique()[0]


def get_consumo_co2(tipo_veiculo, km_rodados):

    consumo_l = get_fator_consumo(tipo_veiculo)
    consumo_fato = consumo_l * km_rodados

    fator_emissao_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]
    fator_emissao_co2_modelo = fator_emissao_modelo['emissa_co2_l_diesel'].unique()[0]
    emissao_co2 = fator_emissao_co2_modelo * consumo_fato

    return emissao_co2

def get_consumo_no2(tipo_veiculo, km_rodados):

    consumo_l = get_fator_consumo(tipo_veiculo)
    consumo_fato = consumo_l * km_rodados

    fator_emissao_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]
    fator_emissao_no2_modelo = fator_emissao_modelo['emissa_no2_l_diesel'].unique()[0]
    emissao_no2 = fator_emissao_no2_modelo * consumo_fato

    return emissao_no2



In [12]:
data_frame = []

for id_onibus in posicoes_df['id_onibus'].unique():
    df_onibus = posicoes_df[posicoes_df['id_onibus'] == id_onibus].reset_index(drop=True)
    df_onibus.sort_values(by='t', inplace=True)
    for i, row in df_onibus.iterrows():
 
        if i < len(df_onibus)-1:
            x_atual = row['x']
            x_depois = df_onibus.loc[i+1, 'x']
            y_atual = row['y']
            y_depois = df_onibus.loc[i+1, 'y']
            momento_inicial = row['t']
            momento_final = df_onibus.loc[i+1, 't']
            tipo_veiculo = get_tipo_onibus(row['id_onibus'])
            litro_disel = get_fator_consumo(tipo_veiculo)
 
            #mas tem que reprojetar para SIRGAS 2000
            dist = (x_depois - x_atual)**2 + (y_depois - y_atual)**2
            
            dados = {
                'distancia_percorrida' : dist,
                'ponto_inicial' : [x_atual, y_atual],
                'ponto_final' : [x_depois, y_depois],
                'momento_inicial' : momento_inicial,
                'momento_final' : momento_final,
                'modelo' : tipo_veiculo,
                'litro_disel_km' : litro_disel,
                'emissao_no2' : get_consumo_no2(tipo_veiculo, dist),
                'emissao_co2' : get_consumo_co2(tipo_veiculo, dist)
            }

            data_frame.append(dados)

df_final = pd.DataFrame(data_frame)

In [13]:
df_final

,distancia_percorrida,ponto_inicial,ponto_final,momento_inicial,momento_final,modelo,litro_disel_km,emissao_no2,emissao_co2
0,0.001954,"[-46.62238437490767, -23.545053610060105]","[-46.61720457905448, -23.50114931626892]",2025-08-12T11:37:04.509539,2025-08-12T11:45:04.509539,Biarticulado,0.90,0.000048,0.000138
1,0.004156,"[-46.58485651555318, -23.560790542650523]","[-46.61310254123852, -23.50284458625373]",2025-08-12T11:55:04.509539,2025-08-12T11:57:04.509539,Biarticulado,0.90,0.000101,0.000292
2,0.009996,"[-46.61310254123852, -23.50284458625373]","[-46.55507713598234, -23.584261856621684]",2025-08-12T11:57:04.509539,2025-08-12T12:31:04.509539,Biarticulado,0.90,0.000244,0.000703
3,0.000479,"[-46.591957231709806, -23.540093384219475]","[-46.58485651555318, -23.560790542650523]",2025-08-12T12:22:04.509539,2025-08-12T11:55:04.509539,Biarticulado,0.90,0.000012,0.000034
4,0.006068,"[-46.55507713598234, -23.584261856621684]","[-46.62238437490767, -23.545053610060105]",2025-08-12T12:31:04.509539,2025-08-12T11:37:04.509539,Biarticulado,0.90,0.000148,0.000427
5,0.006813,"[-46.563798800372, -23.528433857285663]","[-46.64633537264458, -23.52941963277402]",2025-08-12T11:52:04.509539,2025-08-12T12:04:04.509539,Midiônibus,0.47,0.000068,0.000286
6,0.000206,"[-46.64633537264458, -23.52941963277402]","[-46.64849495800811, -23.54362034288309]",2025-08-12T12:04:04.509539,2025-08-12T12:23:04.509539,Midiônibus,0.47,0.000002,0.000009
7,0.002611,"[-46.63862806646152, -23.54586535485801]","[-46.61715914537871, -23.592236366792434]",2025-08-12T12:04:04.509539,2025-08-12T11:53:04.509539,Midiônibus,0.47,0.000026,0.000110
8,0.000102,"[-46.64849495800811, -23.54362034288309]","[-46.63862806646152, -23.54586535485801]",2025-08-12T12:23:04.509539,2025-08-12T12:04:04.509539,Midiônibus,0.47,0.000001,0.000004
9,0.000295,"[-46.55287069729631, -23.515171019362825]","[-46.563798800372, -23.528433857285663]",2025-08-12T12:26:04.509539,2025-08-12T11:52:04.509539,Midiônibus,0.47,0.000003,0.000012


In [14]:
save_csv(df_final, "df_final.csv")

Base salva em data\df_final.csv
